In [0]:
####How do i apply the schema from another excel 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Create DataFrame and Insert Data") \
    .getOrCreate()

In [0]:
#File uploaded to /FileStore/Youtube_videos_list.txt
df=spark.read.format("text")\
    .option("header","false")\
        .load("/FileStore/Youtube_videos_list.txt")

columns=["Sno","Descirption","Links"]
#df.toDF(*columns)

In [0]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Schema Comparison") \
    .getOrCreate()

# Define data for the first DataFrame
data1 = [
    (1, "John Doe", "Engineering", 100000,100),
    (2, "Jane Smith", "Marketing", 95000,101),
    (3, "Sam Brown", "Sales", 80000,102)
]

# Define schema for the first DataFrame
schema1 = ["EmployeeID", "Name", "Department", "Salary","Dept ID"]

# Define data for the second DataFrame
data2 = [
    (1, "John Doe", "Engineering", 100000, "Full-time"),
    (2, "Jane Smith", "Marketing", 95000, "Full-time"),
    (3, "Alice Green", "HR", 90000, "Part-time")
]

# Define schema for the second DataFrame
schema2 = ["EmployeeID", "Name", "Department", "Salary", "EmploymentType"]

# Create the first DataFrame
df1 = spark.createDataFrame(data1, schema=schema1)

# Create the second DataFrame
df2 = spark.createDataFrame(data2, schema=schema2)

####list of the columns Missing in  DF_2  (Present in DF_1)

In [0]:
list(set(df1.columns)-set(df2.columns))

Out[5]: ['Dept ID']

####list of the columns Missing in  DF_1  (Present in DF_2)

In [0]:
list(set(df2.columns)-set(df1.columns))

Out[6]: ['EmploymentType']

####Ask is to make a one dataframe by union - Dynamic code


In [0]:
# Function to align DataFrames by adding missing columns with null values
if df1.columns!=df2.columns:
    unique_columns=list(set(df1.columns+df2.columns))
    for col in unique_columns:
        if col not in df1.columns:
            df1=df1.withColumn(col,lit(None))
        elif col not in df2.columns:
            df2=df2.withColumn(col,lit(None))

In [0]:
print(df1.columns)

['EmployeeID', 'Name', 'Department', 'Salary', 'Dept ID', 'EmploymentType']


In [0]:
print(df2.columns)

['EmployeeID', 'Name', 'Department', 'Salary', 'EmploymentType', 'Dept ID']


####Column Names & DATA TYPE CHECK 

In [0]:
# Define data and schema for the master DataFrame
master_data = [
    (1, "John Doe", "Engineering", 100000, "Full-time"),
    (2, "Jane Smith", "Marketing", 95000, "Full-time")
]

master_schema = StructType([
    StructField("EmployeeID", IntegerType(), True),
    StructField("FullName", StringType(), True),
    StructField("Department", StringType(), True),
    StructField("Salary", IntegerType(), True),
    StructField("EmploymentType", StringType(), True)
])

# Create the master DataFrame
master_df = spark.createDataFrame(master_data, schema=master_schema)


# Define data and schema for another DataFrame with a different schema
data = [
    (1, "John Doe", "Engineering", 100000),
    (2, "Jane Smith", "Marketing", 95000),
    (3, "Sam Brown", "Sales", "80000")
]

schema = StructType([
    StructField("EmpID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Dept", StringType(), True),
    StructField("Salary", StringType(), True)  # Note the different type here
])

# Create the DataFrame
df = spark.createDataFrame(data, schema=schema)



In [0]:
 #master_df is the master dataframe which we will refer to check the datatype,name 
 #We will allign all the upcoming data according to master_df datatype,name & df is up stream data 
 master_schema = master_df.schema

####Rename Column Names and Data Types Dynamically
def align_with_master(df, master_df):
    # Get the schema of the master DataFrame
    master_schema = master_df.schema
    
    # Create a mapping from the master schema
    master_columns = {field.name: field.dataType for field in master_schema.fields}
    
    # Rename columns to match the master DataFrame
    for old_name, new_name in zip(df.columns, master_df.columns):
        if old_name not in master_columns:
            df = df.withColumnRenamed(old_name, new_name)
    
    # Ensure the columns are in the same order and types as the master DataFrame
    for field in master_schema.fields:
        if field.name not in df.columns:
            # Add missing columns with null values
            df = df.withColumn(field.name, lit(None).cast(field.dataType))
        else:
            # Cast columns to the correct data type
            df = df.withColumn(field.name, col(field.name).cast(field.dataType))
    
    # Select the columns in the same order as the master DataFrame
    df = df.select([field.name for field in master_schema.fields])
    
    return df


In [0]:
df.show()

+-----+----------+-----------+------+
|EmpID|      Name|       Dept|Salary|
+-----+----------+-----------+------+
|    1|  John Doe|Engineering|100000|
|    2|Jane Smith|  Marketing| 95000|
|    3| Sam Brown|      Sales| 80000|
+-----+----------+-----------+------+

